In [ ]:
### Pandas понадобится нам для работы с данными
import pandas as pd
### Различные подмодули из rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem.Draw import IPythonConsole
### для сохранения в файл
import pickle
### для работы с файловой системой
import os
### для отрисовки
from matplotlib import pyplot as plt
from matplotlib.image import NonUniformImage
### Для работы с массивами
import numpy as np

### Переменная, указывающая папку с нашими файлами
data_folder = "/home/alex/ml_lectures"

In [ ]:
df1 = pd.read_csv(os.path.join(data_folder,"bace.csv"))
### Оставляем только столбы, которые нас интересуют
df = df1[["smiles", "Class", "Model", "pIC50", "AlogP"]].copy()
### Хорошим тоном будет чистить за собой и не захломлять память
del df1
### Простой вызов DataFrame позволяет красиво отрисовывать таблицу с помощью matplotlib
df

In [ ]:
### При анализе стоит внимательно смотреть на данные
### тут можно и построчно вызывать df["pIC50"].min(), df["pIC50"].max
### но циклом элегантнее
from pandas import Series as ps
print("Average data for pIC50")
for function,name in zip( [ps.min, ps.max, ps.mean, ps.median], ["min", "max", "mean", "median"] ):
    print("{:>10} is {:>5.2f}.".format(name, function(df["pIC50"])))
df["pIC50"].hist(bins=20)

#### __2.3 Чтение из SDF файла__
Может случиться и так, что вам необходимо прочитать данные  
из файла с 2D/3D структурами и фичами. 
RDKit предоставляет простой способ чтения SDF файлов

In [ ]:
### Взглянем на структуру SDF файла
### 1-ой строкой идет параметр _Name. 
### В нашем случае он совпадает со значением в столбце smiles
### Далее идут блоки атомов и связей, а затем блок доп фичей
with open(os.path.join(data_folder,"bace.sdf")) as fsdf:
    lines = fsdf.readlines()
for line in lines[:170]:
    print(line, end="")
del lines

In [ ]:
### Будем пользоваться Chem.SDMolSupplier
### Если забыли или не знали как пользоваться той или иной функцией
### Информацию по объектам можно получить 2 способами
### Для вызова справки расскоментруйте строки ниже
# help(Chem.SDMolSupplier) # Python-like
# ?Chem.SDMolSupplier # Jupyter-like

In [ ]:
### Итератор по молекулам внутри SDF файла
suppl = Chem.SDMolSupplier(os.path.join(data_folder,"bace.sdf"))
### 1. Собираем все названия сохраненных фичей по всем молекулам
###    В нашем случае у каждой молекулы ровно 5 фичей, 
###    но в общем случае там могут быть пропуски
S = set(["_Name"])
for m in suppl: S.update(set(m.GetPropNames()))
print("Список всех фичей:", S)

### Создаем словарь с фичами и заполняем его
MolDict = { s:[] for s in S}
for m in suppl:
    for feature in S:
        MolDict[feature].append(m.GetProp(feature))

In [ ]:
### Генерируем таблицу на базе словаря
df_from_sdf = pd.DataFrame(MolDict)
df_from_sdf

In [ ]:
df_from_sdf.info()

#### __2.4 Канонизация SMILES__

__Какие мы видим различия с исходной таблицей?__
* _Name нужно переименовать в smiles
* Dtype автоматически не преобразовались, нужно менять

In [ ]:
### переименуем колонку
df_from_sdf.rename(columns={"_Name":"smiles"}, inplace=True)
### меняем dtype ровно на те, что указаны в df
### если бы были проблемы с конвертацией, интерпретатор бы выругался
df_from_sdf = df_from_sdf.astype(dict(df.dtypes))

In [ ]:
### Проверяем таблицы на равенство
if df_from_sdf.equals(df[list(df_from_sdf)]): print("\nМатрицы из csv и sdf идентичны ") 
else: print("Матрицы не равны")
### Матрицы не равны.
### Отметим, что для проверки равенства нужно выровнять столбцы
### Мы сделали это указав таблицу в виде df[list(df_from_sdf)],
### т.е. просим представить df с порядком столбцов как в df_from_sdf
### Посмотрим, что не совпадает подробнее
df_from_sdf.eq(df[list(df_from_sdf)])

In [ ]:
### Посмотрим на первую строку smiles
print(df_from_sdf["smiles"][0])
print(df["smiles"][0])

__Мы видим классическую ситуацию неравенства smiles__  
Можно было бы предположить, что молекулы разные  
Но для начала давайте переведем все в канонический вид

In [ ]:
### Переводим smiles к каноническому виду
df.loc[:,"smiles"] = df["smiles"].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x))).copy()
df_from_sdf.loc[:,"smiles"] = df_from_sdf["smiles"].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x))).copy()

In [ ]:
if df_from_sdf.equals(df[list(df_from_sdf)]): print("\nМатрицы из csv и sdf файла идентичны ") 
else: print("Матрицы не равны"); del df_from_sdf

In [ ]:
### Еще раз взглянем на то, что получилось
df

### 3. Генерация фичей и визуализация

In [ ]:
### Генерируем молекуклярные обекты RDKit из SMILES
df["rdkit_mol"] = [ Chem.MolFromSmiles(i) for i in df["smiles"]]
### Рассчитываем молекулярную массу, число неводородных атомов и канонический smiles
### исходя из молекулярного объекта
df["MW"] = [ Descriptors.MolWt(m) for m in df["rdkit_mol"]]
df["NA"] = [ m.GetNumAtoms() for m in df["rdkit_mol"]]
### Смотрим, что получилось
df

In [ ]:
### Глянем на распределение масс
from pandas import Series as ps
print("Average data for MW")
for function,name in zip( [ps.min, ps.max, ps.mean, ps.median], ["min", "max", "mean", "median"] ):
    print("{:>10} is {:>5.2f}.".format(name, function(df["MW"])))
df["MW"].hist(bins=20)

In [ ]:
### Сортируем таблицу по значению в столбце MW (молекулярная масса)
### в убывающем порядке и оставляем первые 10 молекул
arr = df.sort_values(by="MW", ascending=False).iloc[:10]
img=Draw.MolsToGridImage(arr['rdkit_mol'].values, molsPerRow=2,subImgSize=(600,450), 
                         maxMols=999999, 
                         legends=["MW: {:.1f}".format(x,y)for x,y in zip(arr['MW'].values,arr['smiles'].values)])
img

In [ ]:
### Для надежности, давайте запишем эти молекулы в sdf
### И глянем на них в визуализаторе
### Сейчас они в 2D и без водородов
### 1 шаг. Добавляем водороды
mols = [Chem.AddHs(m) for m in arr['rdkit_mol']]
### 2 шаг. Генерируем 3D-структуры.
for m in mols:
    AllChem.EmbedMolecule(m)
### 3 шаг. Открываем файл для записи и записываем молекулы по порядку
###  задав при этом параметр _Name значением smiles из df. 
### В sdf можно записывать различные параметры молекул
with Chem.SDWriter(os.path.join(data_folder,'heavy_mols.sdf')) as w:
    for i in range(len(mols)):
        m = mols[i]
        m.SetProp("_Name",str(arr['smiles'].iloc[i]))
        for prop in ["Class", "Model", "pIC50", "AlogP"]:
            m.SetProp(prop,str(arr[prop].iloc[i]))
        w.write(m)

In [ ]:
### Давайте взглянем посмотрим на эти молекулы
### С помощью интерактивного визуализатора
import py3Dmol
from ipywidgets import interact,fixed,IntSlider

def MolTo3DView(mol, size=(800, 600), style="stick", surface=False, opacity=0.5):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

def mol_idx(idx):
    mol = mols[idx]
    return MolTo3DView(mol, style='stick').show()
    
_ = interact(mol_idx, idx=IntSlider(min=0,max=len(mols)-1, step=1))


In [ ]:
### Сортируем таблицу по значению в столбце MW (молекулярная масса) и оставляем первые 10 молекул
arr = df.sort_values(by="MW").iloc[:10]
### Отрисовываем эти 10 молекул
img=Draw.MolsToGridImage(arr['rdkit_mol'].values, molsPerRow=2,subImgSize=(500,350),
                         ### Обратите внимание на удобную функцию формат, логикой похожую на printf
                         ### zip генерирует tuple итератор на оснрове указаанных массивов
                         legends=["MW: {:.1f}  smi: {}".format(x,y)for x,y in zip(arr['MW'].values,arr['smiles'].values)])
img

In [ ]:
### Посмотрим на конретный хемотип "c1c(O)ccc(CCN)c1"
### В первых 200 по массе молекулах
mols = df.sort_values(by="MW").iloc[:200]['rdkit_mol'].values
p = Chem.MolFromSmarts("c1c(O)ccc(CCN)c1")
subms = [x for x in mols if x.HasSubstructMatch(p)]
### Выравнеем визуализацию по совпадающей подструктуре
AllChem.Compute2DCoords(p)
for m in subms:
    _ = AllChem.GenerateDepictionMatching2DStructure(m,p)
### Отрисовываем
img=Draw.MolsToGridImage(subms,molsPerRow=2,subImgSize=(550,200)) 
img

In [ ]:
### Давайте теперь подсветим общую подструктуру
### Функция для отметки общих атомов
def higlight_common(mols):
    ### находит общую подструктуру для массива молекул
    ### Но мы и так ее знаем ))) "c1c(O)ccc(CCN)c1"
    mcs = rdFMCS.FindMCS(mols)
    mcs_mol = Chem.MolFromSmarts(mcs.smartsString)
    target = []
    ### Проходимся по молекулам и отмечаем атомы
    for i in mols:
        match = i.GetSubstructMatch(mcs_mol)
        target_buf=[]
        for atom in i.GetAtoms():
            if atom.GetIdx() in match:
                target_buf.append(atom.GetIdx())
        target.append(target_buf)
    ### Возвращаем массив отмеченных атомов
    return target
target = higlight_common(subms)
### Отрисовываем
### Отметим, что выравнивание мы сделали на прошлом шаге
img=Draw.MolsToGridImage(subms,molsPerRow=2,subImgSize=(550,200), highlightAtomLists=target) 
img

### 4. Расчет доноров и акцепторов. SMARTS

In [ ]:
### Посмотрим на конретный хемотип пиридин с амидной группой в любом положении кольца
### При этом должна быть любая тройная связь
mols = df.sort_values(by="MW")['rdkit_mol'].values
p1 = Chem.MolFromSmarts("[$(c1nccc(C(=O)N)c1),$(c1cncc(C(=O)N)c1),$(c1ccnc(C(=O)N)c1)]")
p2 = Chem.MolFromSmarts("*#*")
subms = [x for x in mols if x.HasSubstructMatch(p1) and x.HasSubstructMatch(p2)]
### Выравнеем визуализацию по совпадающей подструктуре
mcs = rdFMCS.FindMCS(subms)
mcs_mol = Chem.MolFromSmarts(mcs.smartsString)
AllChem.Compute2DCoords(mcs_mol)
for m in subms:
    _ = AllChem.GenerateDepictionMatching2DStructure(m,mcs_mol)
target = higlight_common(subms)
### Отрисовываем
IPythonConsole.drawOptions.addAtomIndices = True
img=Draw.MolsToGridImage(subms,maxMols=9999999,molsPerRow=2,subImgSize=(500,400), highlightAtomLists=target) 
img
### Как видим, общее совпадение оказалось куда более емким

In [ ]:
### Расчитаем количество центров--доноров водородной связи
### Смартс для донора -- любой гетероатом с ненулевым количеством водородов и с неотрицательным зарядом
HDon_SMARTS = "[!$([#6,H0,-,-2,-3])]"
### Генерируем RDKit молекулы из SMARTS-строки
HDon_mol = Chem.MolFromSmarts(HDon_SMARTS)
HDon_count = pd.Series(df['rdkit_mol'].apply(lambda x: len(x.GetSubstructMatches(HDon_mol))))

In [ ]:
### С помощью IPythonConsole можно контролировать настройки отрисовки
### Меняем размер изображения
IPythonConsole.molSize = 500,500
### функция GetSubstructMatches автоматически помечает выбранные атомы
### Молекула отобразится с помеченными донорами
df['rdkit_mol'].iloc[1]

In [ ]:
### Сравним наши расчеты со стандартным вариантом из RDKit
HDon_count_rdkit = pd.Series([ Chem.Lipinski.NumHDonors(m) for m in df['rdkit_mol'] ])
print("Сумма разности модулей двух вариантов расчета доноров: {}".format((HDon_count - HDon_count_rdkit).abs().sum()))

In [ ]:
### Расчитаем количество центров--aкцепторов водородной связи
HAcc_SMARTS = "[!$([#6,F,Cl,Br,I,o,s,nX3,#7v5,#15v5,#16v4,#16v6,*+1,*+2,*+3])]"
### Акцептор Н-связи представляет собой гетероатом без положительного заряда, 
### Обратите внимание, что включены отрицательно заряженные кислород или сера. 
### Исключаются галогены, включая F, гетероароматический кислород, сера и пиррол N. 
### Исключаются более высокие уровни окисления N, P, S. 
### P(III) включен. Стоит ли исключить (O=S =O)?
### Генерируем RDKit молекулы из SMARTS-строки
HAcc_mol = Chem.MolFromSmarts(HAcc_SMARTS)
HAcc_count = pd.Series(df['rdkit_mol'].apply(lambda x: len(x.GetSubstructMatches(HAcc_mol))))

In [ ]:
df['rdkit_mol'].iloc[1]

In [ ]:
### Сравним наши расчеты со стандартным вариантом из RDKit
HAcc_count_rdkit = pd.Series([ Chem.Lipinski.NumHAcceptors(m) for m in df['rdkit_mol'] ])
print("Сумма разности модулей двух вариантов расчета акцепторов: {}".format((HAcc_count - HAcc_count_rdkit).abs().sum()))

In [ ]:
### Расчитаем количество центров--aкцепторов водородной связи
HAcc_SMARTS = '[$([O,S;H1;v2]-[!$(*=[O,N,P,S])]),' + \
               '$([O,S;H0;v2]),$([O,S;-]),$([N;v3;!$(N-*=!@[O,N,P,S])]),' \
                + '$([nH0,o,s;+0])]'
### Найдено в репозитории RDKit
### Генерируем RDKit молекулы из SMARTS-строки
HAcc_mol = Chem.MolFromSmarts(HAcc_SMARTS)
HAcc_count = pd.Series(df['rdkit_mol'].apply(lambda x: len(x.GetSubstructMatches(HAcc_mol))))
### Сравним наши расчеты со стандартным вариантом из RDKit
print("Сумма разности модулей двух вариантов расчета акцепторов: {}".format((HAcc_count - HAcc_count_rdkit).abs().sum()))
df['rdkit_mol'].iloc[1]

In [ ]:
df["HDon"] = HDon_count
df["HAcc"] = HAcc_count

In [ ]:
_ = df["HDon"].hist(bins=15)

In [ ]:
_ = df["HAcc"].hist(bins=15)

### 5. Правило Липинского

In [ ]:
### Напишем функцию для отображения 2D гистограмм
### Квантили нужны, чтобы не отображать пустое пространство. Попробуйте quantile=1e-16
def plot2Dhist(df, col1, col2, bins=20, quantile=0.01):
    H, xedges, yedges = np.histogram2d(df[col1].values, df[col2].values, bins=bins, 
                                       range=[df[col1].quantile([quantile, 1-quantile], interpolation='nearest'),
                                              df[col2].quantile([quantile, 1-quantile], interpolation='nearest') ] )
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, title=col1+"(x) - "+col2+"(y)")
    plt.imshow(H.T, interpolation='nearest', origin='lower', aspect='auto', cmap='plasma',
            extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])

In [ ]:
plot2Dhist(df, "pIC50", "HAcc", bins=[40,19], quantile=1e-20)

In [ ]:
plot2Dhist(df, "pIC50", "HDon", bins=[40,16], quantile=1e-20)

In [ ]:
plot2Dhist(df, "pIC50", "MW", bins=[40,50], quantile=1e-20)

In [ ]:
plot2Dhist(df, "pIC50", "AlogP", bins=[40,40], quantile=1e-20)

### 6. Визуализация Белка

In [ ]:
### А что же за белок такой.
### Посмотрим на него ИНТЕРАКТИВНО
with open(os.path.join(data_folder,"4ivt.pdb")) as ifile:
    system = "".join([x for x in ifile])
view = py3Dmol.view(width=600, height=600)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()

In [ ]:
### Примитивным перебором выберем пролины и 
### небелковую неводную часть

view = py3Dmol.view(width=600, height=600)
view.addModelsAsFrames(system)
i = 0
for line in system.split("\n"):
    split = line.split()
    style="cartoon"
    if len(split) == 0:
        continue
    if split[0] != "ATOM":
        if split[0] == "HETATM" and split[3]!='HOH':
                style="sphere"
                color="spectrum"
        else:
            continue
    else:
        if split[3] == "PRO":
            color = "red"
        else:
            color = "yellow"
    idx = int(split[1])
    view.setStyle({'model': -1, 'serial': i+1}, {style: {'color': color}})
    i += 1
view.zoomTo()
view.show()

In [ ]:
### Более раумное чтение в классы атомов и молекул
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["resid"] = int(int(line[22:26]))
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip()

    def __str__(self):
        line = list(" " * 80)

        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].ljust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].ljust(3)
        line[22:26] = str(self["resid"]).ljust(4)
        line[30:38] = str(self["x"]).rjust(8)
        line[38:46] = str(self["y"]).rjust(8)
        line[46:54] = str(self["z"]).rjust(8)
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"
class Molecule(list):
    def __init__(self, file):
        for line in file:
            if "ATOM" in line[:4] or "HETATM" in line[:6]:
                self.append(Atom(line))

    def __str__(self):
        outstr = ""
        for at in self:
            outstr += str(at)
        return outstr

with open(os.path.join(data_folder,"4ivt.pdb")) as ifile:
    mol = Molecule(ifile)

In [ ]:
### Умное выделение
for at in mol:
    if at["resname"] == "PRO":
        at["pymol"] = {"stick": {'color': "red"}}
    elif at["resname"] == "GLY":
        at["pymol"] = {"stick": {'color': 'blue'}}
    elif (at["type"] == "HETATM") and (at["resname"] != "HOH") :
        at["pymol"] = {"sphere": {'color': 'spectrum'}}
view = py3Dmol.view(width=600, height=500)
view.addModelsAsFrames(str(mol))
for i, at in enumerate(mol):
    default = {"line": {'color': 'black'}}
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
view.zoomTo()
view.show()